# Unweighted Graphs:


https://learning.ecam.be/SA4T/slides/04-graphs

A graph is a set of vertices and edges.

<img src="img/graphs.png" width="35%">



### Adjacency List :

## Graph Exploration:

<img src="img/bfs_dfs.png" width="35%">

### BFS :


ex. Shortest-Path algorithms


In [6]:
# Go through all vertices ? why

import collections
def BFS(adj, start):
    # queue = [start]  # queue of next nodes to visit
    queue = collections.deque([start])
    visited = set()  # already visited nodes/

    while queue:
        node = queue.pop(0)         # take first nodes
        # node = queue.popleft()    #! why not work ?
        print("Visiting: ", node)
        visited.add(node)
        queue.extend(adj[node] - visited) # add neighbor adjacent nodes to visit next
    

BFS(
    { 
        0: {1, 2}, 
        1: {3, 4}, 
        2: {5, 6}, 
        3: set(), 4: set(), 5: set(), 6: set() 
    }, 
        0
    )


# Complexity : O(V + E)
    # While : O(V) = O(Vertices)
    # queue.ext : O(E) go through all edges (aretes) starting at node

# Exam : expliqu O(V + E)



NameError: name 'collections' is not defined

### DFS :

- One-solution algorithm. 

- Backtracking.

ex. Sudoku

<img src="img/tree.png" width="35%">

In [7]:


def DFS(adj: dict[any, set]):
    visited = set()


    def explore(start):
        visited.add(start)
        print(f"Visiting: {start}")
        for neighbor in adj[start] - visited:
            explore(neighbor)
        print(f"Finish exploring: {start}")

    # explore(0)   # 0, 1, 3, 4, 2, 5, 6: 

    for node in adj:
        if node not in visited:
            explore(node)

DFS(
    { 
        0: {1, 2}, 
        1: {3, 4}, 
        2: {5, 6}, 
        3: set(), 4: set(), 5: set(), 6: set() 
    })






Visiting: 0
Visiting: 1
Visiting: 3
Finish exploring: 3
Visiting: 4
Finish exploring: 4
Finish exploring: 1
Visiting: 2
Visiting: 5
Finish exploring: 5
Visiting: 6
Finish exploring: 6
Finish exploring: 2
Finish exploring: 0


### X1 : modify BFS to get shortest Path

In [13]:
# copy bfs and add a dict to track

import collections

def shortest_paths(adj, start):
    dist = {start: 0}
    # visited = set()  # can remove ?
    queue = collections.deque([start])
    
    while queue:
        node = queue.popleft()
        print("Visiting: ", node)
        # visited.add(node)
        for neighbor in adj[node] - set(dist.keys()):
            if neighbor not in dist:
                dist[neighbor] = dist[node] + 1
                queue.append(neighbor)
    print(dist)

shortest_paths(
    { 
        0: {1, 2}, 
        1: {3, 4}, 
        2: {5, 6}, 
        3: set(), 4: set(), 5: set(), 6: set() 
    }, 
        0
    )

# step by step:
    # queue = [0]
    # node = 0, queue = []
    # visited = {0}
    # queue = [1,2]    # whats the distance to 1 and 2 ?




Visiting:  0
Visiting:  1
Visiting:  2
Visiting:  3
Visiting:  4
Visiting:  5
Visiting:  6
{0: 0, 1: 1, 2: 1, 3: 2, 4: 2, 5: 2, 6: 2}


### x2 : Sudoku

- A grid will be represented by an array of 81 integers between `0` and `9` We use `0` to indicate that the entry is empty.
- An edge exists between two grids if they differ by only one number, and they both satisfy the sudoku rules (they are not necessarily solvable). 
- Explore the graph until you get a solve grid. If it's impossible, `backtrack`.


- DFS : because single solution, dont want to explore all possibilities

In [ ]:

# def blacklist(grid: list[int], n: int) -> set[int]:
#     """ 
#         Return numbers that cannot be placed at position n
#         (because they already appear in same row, column, or 3x3 box)
#         For entry n,
#         specify whuch numbers cannot be used,
#         bc they have been used in the row, column 
#         grid = [1, 0, 0,...] => blacklist(grid, 1) = 1
#     """
#     blacklisted = set()
#     # Find set of numbers already in the row, add them to res
#     row_number = grid[n//9]
#     row_vals = {i for i in grid[row_number * 9 + 9] if v != 0}

#     # Find // columns
#     column_number = grid[n%9]
#     column_vals = {grid[column_number + 9*i] for i in range(9) if i != 0}

#     # Find // in 3x3 box (choose top-left as reference):
#     box_vals = set()
#     box_row = (row_number //3) * 3
#     box_col = (column_number // 3) * 3
#     for next_right in (0,1,2) : # get next 3 right and next 3 down
#         for next_down in (0,1,2):
#             val_id =  (box_row + next_right) * 9 + (box_column + next_down)
#             val = grid[val_id]
#             if val != 0: box_vals.add(val)

#     return row_vals | column_vals | box_vals

def pretty_print(grid):
    """Print a flat 81-element sudoku `grid` as a 9x9 board.
    Empty cells (0) are printed as '.'.
    """
    if len(grid) != 81:
        raise ValueError("grid must be length 81")
    for r in range(9):
        # build row as strings (use '.' for empty)
        row = [(str(grid[r*9 + c]) if grid[r*9 + c] != 0 else '.') for c in range(9)]
        # join with vertical separators for 3x3 blocks
        line = ' '.join(row[0:3]) + ' | ' + ' '.join(row[3:6]) + ' | ' + ' '.join(row[6:9])
        print(line)
        # print horizontal separator after every 3 rows (except after last)
        if r % 3 == 2 and r != 8:
            print('-' * 21)

def blacklist_prof(grid: list[int], n: int) -> set[int]:
    i, j = n // 9, n % 9
    # row values (indices 9*i .. 9*i+8)
    row = {grid[9*i + k] for k in range(9)}
    # column values (indices j, j+9, j+18, ...)
    col = {grid[9*k + j] for k in range(9)}
    # 3x3 region: top-left corner of the region
    x, y = (i // 3) * 3, (j // 3) * 3
    region = { grid[9*(x + dx) + (y + dy)] for dx in range(3) for dy in range(3) }
    # union, remove 0 (empty cells)
    return (row | col | region) - {0}


def solve(grid: list[int]):
    """ Explore depth first
        if it worked, return grid
        if not, backtrack
    """
    # Find first 0
    # n = 0
    # for i in grid:
    #     if grid[i] == 0: n = i
    #     if i ==len(grid): return grid  # solved no more 0
    if 0 not in grid : return grid
    n = grid.index(0)
    # Loop over all neighboring grids (same Row)
    for i in range(1, 10):
        if i not in blacklist_prof(grid, n):
            grid[n] = i               # place candidate
            result = solve(grid)      # recurse
            if 0 not in result :      # success, bubble up
                return result         #

    grid[n] = 0               #Failure, backtrack
    return grid


def solve_final(grid: list[int]):
    """ Explore depth first
        if it worked, return grid
        if not, backtrack
    """
    # Find first 0
    if 0 not in grid : 
        return grid
    n = grid.index(0)
    # Loop over all neighboring grids (same Row)
    for i in set(range(1, 10)) - blacklist_prof(grid, n):
        grid[n] = i               # place candidate
        result = solve(grid)      # recurse
        if 0 not in result :      # success, bubble up
            return result         #
        grid[n] = 0               #Failure, backtrack
    return grid




pretty_print(solve2(81*[0]))

# 1 ere Row : [ 1, 2, 3, 4, 0, 6, 7, 8, 9 ] --> 5
# 1 ere Row : [ 1, 2, 3, 4, 5, 6, 7, 8, 9 ]
# 2 eme Row : 



1 2 3 | 4 5 6 | 7 8 9
4 5 6 | 7 8 9 | 1 2 3
7 8 9 | 1 2 3 | 4 5 6
---------------------
2 1 4 | 3 6 5 | 8 9 7
3 6 5 | 8 9 7 | 2 1 4
8 9 7 | 2 1 4 | 3 6 5
---------------------
5 3 1 | 6 4 2 | 9 7 8
6 4 2 | 9 7 8 | 5 3 1
9 7 8 | 5 3 1 | 6 4 2
